# BERT word embedding + XGBoost
## First attempt

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as functional
import matplotlib.pyplot as plt
import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig
import gc
from transformers import BertModel
from sklearn.metrics import roc_auc_score,f1_score
import time
import datetime
from bs4 import BeautifulSoup
import re,string,unicodedata
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize

import tensorflow as tf
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
fake = pd.read_csv('/content/drive/MyDrive/thesis-data/Fake.csv')
true = pd.read_csv('/content/drive/MyDrive/thesis-data/True.csv')

fake["label"] = 0
true["label"] = 1

df = pd.concat([fake, true], ignore_index = True)

df['text'] = df['title'] + " " + df['text']
df.drop(columns=['title', 'date', 'subject'], inplace = True)

In [7]:
nltk.download('stopwords')

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()
#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
    
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

#Apply function on review column
df['text']=df['text'].apply(denoise_text)

<ipython-input-8-6e48383d87b9>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [ ]:
from transformers import BertTokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [9]:
df_original = df.copy()
df = df.sample(frac=1).reset_index(drop=True)[:1000]

In [ ]:
# texts = df.text.values
# labels = df.label.values

text = df['text']
target = df['label']

In [ ]:
def bert_encode(input_text, max_len):
    input_ids = []
    attension_masks = []
    for text in input_text:
        output_dict = tokenizer.encode_plus(
            text, 
            add_special_tokens = True,
            truncation=True,
            max_length = max_len,
            pad_to_max_length = True,
            return_attention_mask = True
        )
        input_ids.append(output_dict['input_ids'])
        attension_masks.append(output_dict['attention_mask'])
    return np.array(input_ids), np.array(attension_masks)

In [ ]:
train_input_ids, train_attention_masks = bert_encode(text, 256)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# Train an XGBoost model using the BERT embeddings and attention mask features
dtrain = xgb.DMatrix(np.concatenate([train_input_ids[:900], train_attention_masks[:900]], axis=1), label=labels[:900])
dtest = xgb.DMatrix(np.concatenate([train_input_ids[901:], train_attention_masks[901:]], axis=1), label=labels[901:])
params = {
    'objective': 'multi:softmax',
    'num_class': len(np.unique(labels[:900])),
    'eval_metric': 'merror',
    'max_depth': 6
}
model = xgb.train(params, dtrain)

# Make predictions on the testing set and evaluate the model
y_pred = model.predict(dtest)
accuracy = accuracy_score(labels[901:], y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8585858585858586


In [ ]:
# parameters = {'objective':['reg:squarederror'],
#               'booster':['gbtree','gblinear'],
#               'learning_rate': [0.1], 
#               'max_depth': [7,10,15,20],
#               'min_child_weight': [10,15,20,25],
#               'colsample_bytree': [0.8, 0.9, 1],
#               'n_estimators': [300,400,500,600],
#               "reg_alpha"   : [0.5,0.2,1],
#               "reg_lambda"  : [2,3,5],
#               "gamma"       : [1,2,3]}

# xgb_model = XGBRegressor(random_state=30)

# grid_obj_xgb = RandomizedSearchCV(xgb_model,parameters, cv=5,n_iter=15,scoring='neg_mean_absolute_error',verbose=5,n_jobs=12)
# grid_obj_xgb.fit(df_train, y_train,verbose = 1)

# y_pred_train = grid_obj_xgb.predict(df_train)
# y_pred_test = grid_obj_xgb.predict(df_test)

# err_xgb_train=mean_absolute_error(y_train, y_pred_train, multioutput='raw_values')

In [ ]:
# balanced_accuracy_list = []
# for train, test in sKFold.split(X_train, y_train):
#     pipeline = make_pipeline(SMOTETomek(tomek=TomekLinks(sampling_strategy='majority')), rcv_tree)
#     model = pipeline.fit(X_train[train], y_train[train])
#     best_tree = rcv_tree.best_estimator_
#     prediction = best_tree.predict(X_train[test])
#     balanced_accuracy_list.append(balanced_accuracy_score(orgytrain[test], prediction))
# print("DecisionTreeClassifier balanced accuracy: {}".format(np.mean(balanced_accuracy_list)))

In [16]:
import pandas as pd
import numpy as np
import xgboost as xgb
import random
import transformers
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score


class XGBClassifierWithBERTWrapper:
    
    def __init__(self, params=None, n_iter=50, cv=5, random_state=None):
        self.params = params
        self.n_iter = n_iter
        self.cv = cv
        self.random_state = random_state
        self.tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
        self.kfolds = StratifiedKFold(n_splits=self.cv, shuffle=True, random_state=self.random_state)
        self.estimator = xgb.XGBClassifier()
        self.search = RandomizedSearchCV(estimator=self.estimator, param_distributions=self.params, 
                                         n_iter=self.n_iter, cv=self.kfolds, scoring='roc_auc', 
                                         n_jobs=-1, verbose=3, random_state=self.random_state)
    
    def tokenize(self, X):
        tokenized_X = []
        max_length = 0
        for text in X:
            input_ids = torch.tensor(self.tokenizer.encode(text, add_special_tokens=True))
            tokenized_X.append(input_ids)
            max_length = max(max_length, len(input_ids))
        padded_X = []
        for input_ids in tokenized_X:
            padded_X.append(torch.cat([input_ids, torch.zeros(max_length-len(input_ids), dtype=torch.long)]))
        return padded_X
    
    def train(self, X, y):
        tokenized_X = self.tokenize(X)
        folds_scores = []
        for fold, (train_idx, test_idx) in enumerate(self.kfolds.split(tokenized_X, y)):
            train_X = [tokenized_X[idx] for idx in train_idx]
            train_y = y[train_idx]
            test_X = [tokenized_X[idx] for idx in test_idx]
            test_y = y[test_idx]
            train_X = [torch.cat(train_X, dim=0).squeeze()]
            test_X = [torch.cat(test_X, dim=0).squeeze()]
            self.estimator.fit(train_X, train_y)
            preds = self.estimator.predict(test_X)
            score = roc_auc_score(test_y, preds)
            folds_scores.append(score)
            print(f"Fold {fold+1} ROC AUC score: {score}")
        self.cv_score = np.mean(folds_scores)
        self.search.fit(tokenized_X, y)
        print(f"Best parameters: {self.search.best_params_}")
    
    def predict(self, X):
        tokenized_X = self.tokenize(X)
        tokenized_X = [torch.cat(tokenized_X, dim=0).squeeze()]
        preds = self.estimator.predict(tokenized_X)
        return preds

In [17]:
# create an instance of the wrapper with hyperparameters to search over
params = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200, 500],
    'subsample': [0.5, 0.75, 1],
    'colsample_bytree': [0.5, 0.75, 1],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}
clf = XGBClassifierWithBERTWrapper(params=params, n_iter=50, cv=5, random_state=42)

# load data
#data = pd.read_csv('data.csv')
X = df['text'].values
y = df['label'].values



# train the model
clf.train(X, y)

Token indices sequence length is longer than the specified maximum sequence length for this model (656 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.9/dist-packages/xgboost/data.py:897: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = np.array(data)


ValueError: ignored

In [14]:
clf

In [ ]:
# Make predictions on new data
new_data = pd.read_csv('new_data.csv')
new_X = new_data['text'].values
preds = xgb_wrapper.predict(new_X)

Working wrapper below

In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from transformers import BertTokenizer, TFBertModel
import xgboost as xgb

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Define a function to perform BERT text embedding
def bert_embedding(texts):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens = True,
                            max_length = 64,
                            pad_to_max_length = True,
                            return_attention_mask = True,
                            return_tensors = 'tf'
                       )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = tf.concat(input_ids, axis=0)
    attention_masks = tf.concat(attention_masks, axis=0)

    bert_outputs = bert_model([input_ids, attention_masks])
    pooled_output = bert_outputs[1]
    
    return pooled_output.numpy()

# Define the XGBoost text classifier wrapper
class XGBTextClassifier:
    def __init__(self, params=None):
        self.params = params or {}
        self.clf = None
        
    def fit(self, X, y):
        X_embed = bert_embedding(X)
        self.clf = xgb.XGBClassifier(**self.params)
        self.clf.fit(X_embed, y)
        
    def predict(self, X):
        X_embed = bert_embedding(X)
        y_pred = self.clf.predict(X_embed)
        return y_pred
    
    def predict_proba(self, X):
        X_embed = bert_embedding(X)
        y_pred_proba = self.clf.predict_proba(X_embed)
        return y_pred_proba
    
    def get_params(self, deep=True):
        return self.params
    
    def set_params(self, **params):
        if 'max_depth' in params:
            # Replace 'max_depth' with the appropriate parameter for your version of XGBClassifier
            if 'max_depths' in xgb.XGBClassifier().get_params():
                params['max_depths'] = params.pop('max_depth')
            elif 'max_depth' in xgb.XGBClassifier().get_params():
                params['max_depth'] = params.pop('max_depth')
        self.params.update(params)
        return self

# Define the hyperparameter space for randomized search
params_space = {
    #'learning_rate': np.logspace(-5, 0, 100),
    #'max_depth': np.arange(1, 11),
    #'n_estimators': np.arange(10, 201, 10),
    #'gamma': np.arange(0, 1.1, 0.1)
}

# Load the dataset
#df = pd.read_csv('dataset.csv')
X = df['text'].values
y = df['label'].values

# Define the cross-validation folds
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# Perform randomized search for hyperparameter tuning
xgb_clf = XGBTextClassifier()
random_search = RandomizedSearchCV(xgb_clf, params_space, cv=skf, scoring='accuracy', n_iter=1)
random_search.fit(X, y)

# Print the best hyperparameters and the corresponding accuracy score
print('Best params:', random_search.best_params_)
print('Accuracy score:', random_search.best_score_)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. 

Best params: {}
Accuracy score: 0.883
